In [15]:
# Mengimpor TensorFlow, modul utama untuk pengembangan model machine learning.
import tensorflow as tf
# Mengimpor NumPy, library yang digunakan untuk operasi numerik.
import numpy as np
# Mengimpor os, modul yang digunakan untuk berinteraksi dengan sistem operasi (contoh: membaca file, mengatur direktori, dll.).
import os
# Mengimpor time, modul yang digunakan untuk mengukur waktu atau menunda eksekusi program.
import time

In [16]:
# Menggunakan tf.keras.utils.get_file untuk mengunduh file teks dari URL yang diberikan.
# Nama file yang diunduh adalah 'shakespeare.txt', dan URL sumbernya adalah 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'.
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [17]:
# Load Data

# Membaca isi file teks ('shakespeare.txt') yang telah diunduh dalam mode 'rb' (read binary),
# lalu mendekodekannya dengan encoding 'utf-8' untuk mendapatkan teks dalam bentuk string.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Menghitung panjang teks, yaitu jumlah karakter dalam teks, lalu mencetaknya.
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [18]:
# Mengambil 250 karakter pertama dalam teks dan mencetaknya.
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [19]:
# Menggunakan set(text) untuk mengidentifikasi karakter-karakter unik dalam teks, lalu mengurutkannya dengan sorted.
vocab = sorted(set(text))

# Mencetak jumlah karakter unik yang ditemukan dalam teks.
print(f'{len(vocab)} unique characters')

65 unique characters


In [20]:
# Olah Teks

# Mendefinisikan contoh teks dalam list 'example_texts'.
example_texts = ['abcdefg', 'xyz']

# Menggunakan tf.strings.unicode_split untuk membagi teks dalam contoh teks menjadi karakter-karakter individual.
# input_encoding='UTF-8' menunjukkan bahwa encoding yang digunakan adalah UTF-8.
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Menampilkan hasil pemisahan karakter.
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [21]:
# Membuat lapisan StringLookup dengan parameter-parameter berikut:

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),  # Menggunakan daftar karakter unik sebagai vocabulary.
    mask_token=None  # Tidak ada token mask yang digunakan.
)

In [22]:
# Menggunakan lapisan StringLookup (ids_from_chars) untuk mengonversi karakter-karakter (chars) menjadi ID numerik.
ids = ids_from_chars(chars)

# Menampilkan hasil konversi karakter ke ID.
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [23]:
# Membuat lapisan StringLookup baru dengan parameter-parameter berikut:

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),  # Menggunakan vocabulary dari lapisan ids_from_chars.
    invert=True,  # Mengindikasikan bahwa konversi dari ID ke teks akan dilakukan.
    mask_token=None  # Tidak ada token mask yang digunakan.
)

In [24]:
# Menggunakan lapisan StringLookup (chars_from_ids) untuk mengonversi ID numerik (ids) menjadi karakter teks.
chars = chars_from_ids(ids)

# Menampilkan hasil konversi ID ke karakter teks.
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [25]:
# Menggunakan tf.strings.reduce_join untuk menggabungkan karakter-karakter dalam tensor chars menjadi teks lengkap.
# axis=-1 menunjukkan bahwa penggabungan dilakukan sepanjang sumbu terakhir (axis terakhir).

tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [26]:
# Mendefinisikan sebuah fungsi text_from_ids yang mengambil ID numerik sebagai input.
def text_from_ids(ids):
    # Menggunakan chars_from_ids untuk mengonversi ID menjadi karakter-karakter teks, lalu menggabungkannya menjadi teks lengkap.
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [27]:
# Menggunakan lapisan ids_from_chars untuk mengonversi seluruh teks menjadi ID numerik
# dengan memisahkan teks menjadi karakter-karakter terlebih dahulu.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

# Mengonversi ID numerik menjadi objek tf.data.Dataset.
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

# Mengulangi dataset untuk mengambil 10 data ID dan mencetak karakter teks yang sesuai dengan ID-nya.
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

# Menetapkan panjang urutan (sequence length) menjadi 100.
seq_length = 100

F
i
r
s
t
 
C
i
t
i


In [28]:
# Memisahkan dataset ID numerik menjadi urutan dengan panjang (sequence length) sesuai dengan 'seq_length' + 1.
# drop_remainder=True mengindikasikan bahwa jika ada sisa data yang kurang dari panjang urutan, data tersebut akan dihapus.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Mengambil urutan pertama dari dataset yang telah dibuat dan mencetak karakter teks yang sesuai dengan ID-nya.
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [29]:
# Iterasi melalui lima urutan pertama dalam dataset dan mencetak teks yang sesuai dengan ID-nya.
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

# Membuat fungsi split_input_target yang digunakan untuk membagi urutan menjadi input dan target.
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Membagi urutan dalam list "Tensorflow" menjadi input dan target, kemudian mencetaknya.
split_input_target(list("Tensorflow"))

# Membangun dataset dengan urutan input dan target menggunakan fungsi split_input_target.
dataset = sequences.map(split_input_target)

# Mengambil satu contoh dari dataset, kemudian mencetak input dan targetnya.
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [30]:
# Menetapkan ukuran batch yang akan digunakan dalam pelatihan model.
BATCH_SIZE = 64

# Buffer size digunakan untuk mengatur sejauh mana dataset akan di-shuffle.
# Ini penting karena TensorFlow data dirancang untuk bekerja dengan urutan yang mungkin tak berujung,
# sehingga buffer ini digunakan untuk mengacak elemen-elemen dataset.
BUFFER_SIZE = 10000

# Melakukan operasi shuffle pada dataset dengan menggunakan buffer sebesar BUFFER_SIZE.
# Selanjutnya, dataset dibagi menjadi batch dengan ukuran BATCH_SIZE dan jika ada sisa data yang kurang dari BATCH_SIZE, data tersebut akan dihapus.
# Terakhir, dataset di-prefetch untuk mengoptimalkan kinerja pelatihan.
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

# Menampilkan dataset yang telah diatur.
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [31]:
# Menetapkan ukuran vocab (jumlah kata unik) dari lapisan StringLookup.
vocab_size = len(ids_from_chars.get_vocabulary())

# Menetapkan dimensi embedding yang akan digunakan.
embedding_dim = 256

# Menetapkan jumlah unit dalam lapisan GRU (Gated Recurrent Unit).
rnn_units = 1024

# Membuat kelas model MyModel yang merupakan turunan dari tf.keras.Model.
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  # Metode call digunakan untuk mendefinisikan aliran data dalam model.
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [32]:
# Membuat instance model MyModel dengan parameter-parameter yang telah ditetapkan sebelumnya.
model = MyModel(
    vocab_size=vocab_size,  # Ukuran vocab (jumlah kata unik).
    embedding_dim=embedding_dim,  # Dimensi embedding.
    rnn_units=rnn_units  # Jumlah unit dalam lapisan GRU.
)

In [33]:
# Mengambil satu contoh batch dari dataset.
for input_example_batch, target_example_batch in dataset.take(1):
    # Menggunakan model untuk membuat prediksi dari input contoh batch.
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

# Mencetak ringkasan (summary) model.
model.summary()

# Menggunakan tf.random.categorical untuk menghasilkan indeks karakter berikutnya dari distribusi prediksi.
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

# Mencetak input contoh batch dan prediksi karakter berikutnya.
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

(64, 100, 66) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Input:
 b"d this young prince agree,\nI'll join mine eldest daughter and my joy\nTo him forthwith in holy wedloc"

Next Char Predictions:
 b"!Cxty.YZYBMT--XvyqYfHHFPlJe'FufWOuXQ:tMi:KzfVK[UNK]!kWuE'jsG&?Tg;M$V? pYVKSDZlPiW\n

In [34]:
# Menggunakan SparseCategoricalCrossentropy loss function.
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# Menghitung mean loss dari contoh batch target dan prediksi.
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

# Menghitung e^mean_loss, yang dapat memberikan gambaran lebih intuitif tentang kebaikan model.
tf.exp(example_batch_mean_loss).numpy()

# Mengompilasi model dengan optimizer 'adam' dan loss yang telah ditentukan sebelumnya.
model.compile(optimizer='adam', loss=loss)

# Menentukan direktori tempat checkpoint akan disimpan.
checkpoint_dir = './training_checkpoints'
# Nama file checkpoint.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Membuat callback untuk ModelCheckpoint yang akan menyimpan bobot model pada setiap epoch.
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1909504, shape=(), dtype=float32)


In [ ]:
# Menetapkan jumlah epoch yang akan digunakan dalam pelatihan.
EPOCHS = 20

# Melakukan pelatihan model dengan dataset, dan menggunakan callback untuk menyimpan checkpoint.
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
 19/172 [==>...........................] - ETA: 15:04 - loss: 4.1204

In [ ]:
# Membuat kelas OneStep yang merupakan turunan dari tf.keras.Model.
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Membuat mask untuk mencegah generasi token "[UNK]".
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Memberikan nilai -inf pada setiap indeks yang tidak diinginkan.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Sesuaikan bentuk mask dengan ukuran vocab.
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Mengonversi string menjadi token ID.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Menjalankan model.
    # predicted_logits.shape adalah [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Hanya menggunakan prediksi terakhir.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Menggunakan prediction mask: mencegah generasi token "[UNK]".
    predicted_logits = predicted_logits + self.prediction_mask

    # Mengambil sample dari logits output untuk menghasilkan token ID.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Mengonversi dari token ID menjadi karakter.
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Mengembalikan karakter dan state model.
    return predicted_chars, states

In [ ]:
# Membuat instance model OneStep dengan menggunakan model yang telah dilatih, chars_from_ids, dan ids_from_chars.
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Membuat seed (awal) untuk generasi teks.
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

# Melakukan iterasi untuk menghasilkan teks sepanjang 1000 karakter.
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# Menggabungkan hasil generasi teks.
result = tf.strings.join(result)
end = time.time()

# Mencetak hasil generasi teks dan waktu eksekusi.
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

# Melakukan generasi teks dengan seed yang berbeda.
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

# Melakukan iterasi untuk menghasilkan teks sepanjang 1000 karakter.
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# Menggabungkan hasil generasi teks.
result = tf.strings.join(result)
end = time.time()

# Mencetak hasil generasi teks dan waktu eksekusi.
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

In [ ]:
# Ekspor Model Generator

# Menyimpan model OneStep ke direktori 'one_step'.
tf.saved_model.save(one_step_model, 'one_step')

# Me-reload model yang telah disimpan.
one_step_reloaded = tf.saved_model.load('one_step')

# Inisialisasi state dan seed untuk generasi teks.
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

# Melakukan iterasi untuk menghasilkan teks sepanjang 100 karakter menggunakan model yang di-reload.
for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

# Menggabungkan hasil generasi teks dan mencetaknya.
print(tf.strings.join(result)[0].numpy().decode("utf-8"))